# day 2-2

このノートブックの実行例は[こちら(HTML版)](../notebooks-sample/day-2-2.html)で確認できます

---

## 0. はじめに

ページ上部のメニューバーにある **Kernel** メニューをクリックし、プルダウンメニューから [**Change Kernel ...**] を選び、**gssm2023:Python** を選択してください。

<img src="images/change_kernel1.png" width="30%">

ノートブック上部の右隅に表示されたカーネル名が **gssm2023:Python** になっていることを確認してください。

<img src="images/change_kernel2.png" width="30%">

---

## 3. テキスト解析

### 3.1 形態素解析

#### 3.1.1 MeCab を使う

##### (1) そのまま出力してみる

In [ ]:
import MeCab

tagger = MeCab.Tagger("-r ../tools/usr/local/etc/mecabrc")
print(tagger.parse("今日はいい天気です"))

##### (2) 扱いやすいように Pandas の DataFrame に格納する

In [ ]:
import pandas as pd

node = tagger.parseToNode("今日はいい天気です")
features = []
while node:
    features.append(node.feature.split(','))
    node = node.next

columns = [
    "品詞", 
    "品詞細分類1",
    "品詞細分類2",
    "品詞細分類3",
    "活用型",
    "活用形",
    "基本形",
    "読み",
    "発音",
]
pd.DataFrame(features, columns=columns)

### 3.2 係り受け解析

#### 3.2.1 CaboCha を使う

##### (1) そのまま出力してみる

In [ ]:
import CaboCha

cp = CaboCha.Parser("-r ../tools/usr/local/etc/cabocharc")
tree = cp.parse("今日はいい天気です")
print(tree.toString(CaboCha.FORMAT_LATTICE))

##### (2) ツリー形式で出力する

In [ ]:
print(cp.parseToString("今日はいい天気です"))

##### (3) 係り受けペアを出力する

In [ ]:
# 構文木(tree)からチャンクを取り出す
def get_chunks(tree):
    chunks = {}
    key = 0
    for i in range(tree.size()):
        tok = tree.token(i)
        if tok.chunk:
            chunks[key] = tok.chunk
            key += 1
    return chunks


# チャンク(chunk)から表層形を取り出す
def get_surface(chunk):
    surface = ""
    beg = chunk.token_pos
    end = chunk.token_pos + chunk.token_size
    for i in range(beg, end):
        token = tree.token(i)
        surface += token.surface
    return surface

In [ ]:
tree = cp.parse("今日はいい天気です")
chunks = get_chunks(tree)

for from_chunk in chunks.values():
    if from_chunk.link < 0:
        continue
    to_chunk = chunks[from_chunk.link]

    from_surface = get_surface(from_chunk)
    to_surface = get_surface(to_chunk)

    print(from_surface, '->', to_surface)

---

## 4. データ理解

### 4.1 データのダウンロード (1度だけ実行)

以下のデータをダウンロードします

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2022-2023.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2022/1~2023 GW明け | 本講義の全体を通して使用する |

In [ ]:
# rakuten-1000-2022-2023.xlsx.zip をダウンロードする
FILE_ID = "1n-uvGoH7XQhxexN57hYXuFrkGeHKp-HV"
!gdown --id {FILE_ID}
!unzip rakuten-1000-2022-2023.xlsx.zip

In [ ]:
# # rakuten-1000-2020-2021.xlsx.zip をダウンロードする
# FILE_ID = "1o-9WVPhr0PmSjQE6OCtJbWEVTp5k5Gr4"
# !gdown --id {FILE_ID}
# !unzip rakuten-1000-2020-2021.xlsx.zip

### 4.2 データの読み込み (DataFrame型)

In [ ]:
import pandas as pd

df = pd.read_excel("rakuten-1000-2022-2023.xlsx")
print(df.shape)
display(df.head())

### 4.3 集計

#### (1) エリア別の件数を表示する

In [ ]:
display(df.pivot_table(index=['カテゴリー','エリア'], columns=None, values='コメント', aggfunc='count'))

サンプリングデータなので、すべて1000件になっていることを確認する

#### (2) 投稿者の傾向 (年代別x性別、全体で100%)

In [ ]:
cross_df = pd.crosstab(df['年代'], df['性別'], margins=True, margins_name='合計', normalize=True) * 100
display(cross_df.loc[:,['男性','女性','na','合計']].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

`na` を除いて表示する

In [ ]:
cross_df = pd.crosstab(df['年代'], df['性別'], margins=False, normalize=True) * 100
display(cross_df.loc[cross_df.index!='na',['男性','女性']].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

#### (3) 投稿者の傾向 (性別xカテゴリ別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['性別'], df['カテゴリー'], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['男性','女性','na'],:].style.format('{:.2f}%').bar(axis=None, width=90, height=90, align='left'))

`na` を除いて表示する

In [ ]:
cross_df = pd.crosstab(df['性別'], df['カテゴリー'], margins=False, normalize='columns') * 100
display(cross_df.loc[['男性','女性'],:].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

#### (4) 投稿者の傾向 (性別xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['性別'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['男性','女性','na'],:].style.format('{:.2f}%'))

カラーバーにすると横に長くなるため、ヒートマップでプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[['男性','女性','na'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

`na` を除いてプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,2))
sns.heatmap(cross_df.loc[['男性','女性'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (5) 投稿者の傾向 (年代xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['年代'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df, annot=True, fmt='.2f', cmap='Blues')
plt.show()

`na` を除いてプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[cross_df.index!='na',:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (6) 投稿者の傾向 (同伴者別xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['同伴者'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['一人','家族','恋人','友達','仕事仲間','その他'],:].style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[['一人','家族','恋人','友達','仕事仲間','その他'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (7) 数値評価の構成 (総合別xカテゴリ-エリア別、列ごとに100%)

In [ ]:
cross_df = pd.crosstab(df['総合'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[[5,4,3,2,1],:].style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[[5,4,3,2,1],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (8) 数値評価の平均 (カテゴリ-エリア別x数値評価別)

カテゴリ別

In [ ]:
pivot_df = df.pivot_table(index=['カテゴリー','エリア'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[:,['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

エリア別

In [ ]:
pivot_df = df.pivot_table(index=['カテゴリー'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[:,['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

#### (9)数値評価の平均 (年代x性別)

20~30代

In [ ]:
pivot_df = df[df['年代'].isin(['20代','30代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

40~50代

In [ ]:
pivot_df = df[df['年代'].isin(['40代','50代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

60~80代

In [ ]:
pivot_df = df[df['年代'].isin(['60代','70代','80代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))